In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8442.14it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4635.69it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2571197.34it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8680.29it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4626.89it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 77189.17it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8015.93it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4625.89it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 52856.63it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 9184.01it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4696.32it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 72382.33it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.DRMMTKS()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['mask_value'] = -1
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['top_k'] = 20
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DRMMTKS.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1667400     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_1 (Dot)                     (None, 10, 100)      0           embedding[0

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [10]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=20)
len(train_generator)

102

In [11]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 4s 42ms/step - loss: 0.9129
Validation: normalized_discounted_cumulative_gain@3(0): 0.44913793360110954 - normalized_discounted_cumulative_gain@5(0): 0.5032644544569836 - mean_average_precision(0): 0.47069452226494585
Epoch 2/30
102/102 [==============================] - 3s 33ms/step - loss: 0.7650
Validation: normalized_discounted_cumulative_gain@3(0): 0.5133645680060057 - normalized_discounted_cumulative_gain@5(0): 0.5712097375366855 - mean_average_precision(0): 0.534641819193907
Epoch 3/30
102/102 [==============================] - 3s 33ms/step - loss: 0.6033
Validation: normalized_discounted_cumulative_gain@3(0): 0.599709051518077 - normalized_discounted_cumulative_gain@5(0): 0.6483239888002686 - mean_average_precision(0): 0.6077730086244657
Epoch 4/30
102/102 [==============================] - 3s 33ms/step - loss: 0.4298
Validation: normalized_discounted_cumulative_gain@3(0): 0.6266908483818189 - normalized_discounted_cumulativ